<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_Pacman_Single_Agent_LSTM_030520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''



# Installation and Imports

Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.


In [5]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-b9pphlq3
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-b9pphlq3
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=ce0c74d77add69a6a6e9153d6915d8a7f2ebd63742f8615272f484083c3d1cee
  Stored in directory: /tmp/pip-ephem-wheel-cache-c7ojy4y6/wheels/91/52/02/ec5c530fd76d56a66934ee91abbdae5240b766be1dc176deb7
Successfully built a2c-ppo-acktr
  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-pb_5o95h
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-pb_5o95h
     |████████████████████████████████| 1.6MB 3.4MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=c4cf5d5ac8f856c36b6382639d682092ece7cfbf69476956280b864aa3629b5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-srp7sv0f/wheels/42/1c/91/28314e0c

In [0]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/drive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
import datetime
import time

from benchmark.envs import *
from torch.autograd import Variable

import gym
from benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from custom.EarlyStopping import EarlyStopping_loss
from custom.model import *
from custom.custom_wrappers import DeterminsticNoopResetEnv 
# Imported required for the Model-based RL
#from sklearn.preprocessing import StandardScaler
from baselines.common.atari_wrappers import EpisodicLifeEnv
from torch.nn.utils import clip_grad_norm_


In [0]:
!wandb login ######

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# General Functions

In [0]:
class General_functions():
    def __init__(self, ENV_NAME,n_actions,possible_positions):
        self.ENV_NAME = ENV_NAME
        self.env = gym.make(self.ENV_NAME)
        self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,66))        
        self.env = AtariARIWrapper(self.env)
        self.initial_info_labels = self.env.labels()
    
        self.prev_action_counter =  False
        self.repeated_end = False # Nawid -  Sets self.repeated end as false initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()
        
        self.key_list = ['player_x', 'player_y']


    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i-1] = 1
        return a

    def state_conversion(self,obs,info_labels):
        state = [info_labels[word] for word in self.key_list if word in info_labels]                
        state = np.array(state)
        return state
        
    def next_position(self,state, action):
        next_position = state.copy()
        if action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2

        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            return True
        else:
            return False
            
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action
            else:
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action
    

    
    def check_all_agents(self,info_label,next_info_label):
        repeated = np.equal(info_label,next_info_label).all()
        if repeated:
            self.repeated_end= True

    def check_state(self,state, next_state):
        repeated_state = np.equal(state, next_state).all()
        if repeated_state:
            self.prev_action_counter = False


# Data collection and preprocessing

In [54]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME,n_actions, possible_positions):
        super(Data_collection,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.dataset_random = [] # Where the data is saved each time

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        '''        
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        '''
        #print('D_train shape',D_train.shape)

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.float32)
        #X_train_obs = X_train_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_act = np.array([act for _,_,_,_,act in D_train])
        
        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        y_env_train = y_env_train.astype(np.float32)
        #y_env_train = y_env_train.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        y_env_train = y_env_train - X_train_obs 
        
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.float32)
        #X_val_obs = X_val_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.float32)
        #y_env_val = y_env_val.astype(np.int16)
        y_env_val = y_env_val - X_val_obs 

        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        return env_train_data, env_val_data 

    def setup_dataset(self,env_train_data, env_val_data):
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
        return env_train_data, env_val_data
    
    
    def gather_random_trajectories(self,num_traj):
        #dataset_random = []
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)
            self.env = gym.make(self.ENV_NAME)
            self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,noop_max=66))
            self.env = AtariARIWrapper(self.env)
            obs = self.env.reset()
            
            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(obs, info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action

            while True:
                sampled_action = self.random_action_selection(state,prev_action)
                #sampled_action = np.random.randint(1,5)
                 
                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:
                    self.dataset_random.append([state, next_state, reward, done,sampled_action_one_hot])

                else:
                    self.dataset_random[-1][-2] = True # sets the previous value to done
                    done = True   
                    
                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action
                if done:
                    break    
        
        return self.dataset_random

'''
ENV_NAME = 'MsPacman-ramDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.

NUM_RAND_TRAJECTORIES = 3000
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)
possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
data_collector = Data_collection(ENV_NAME,n_actions,possible_positions)

rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
filename = '/content/drive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
np.save(filename,rand_dataset)

rl_dataset = []
env_train, env_val = data_object.collate_data(data1,rl_dataset)
full_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)
'''

trajectory number: 0
trajectory number: 10
trajectory number: 20
trajectory number: 30
trajectory number: 40
trajectory number: 50
trajectory number: 60
trajectory number: 70
trajectory number: 80
trajectory number: 90
trajectory number: 100
trajectory number: 110
trajectory number: 120
trajectory number: 130
trajectory number: 140
trajectory number: 150
trajectory number: 160
trajectory number: 170
trajectory number: 180
trajectory number: 190
trajectory number: 200
trajectory number: 210
trajectory number: 220
trajectory number: 230
trajectory number: 240
trajectory number: 250
trajectory number: 260
trajectory number: 270
trajectory number: 280
trajectory number: 290
trajectory number: 300
trajectory number: 310
trajectory number: 320
trajectory number: 330
trajectory number: 340
trajectory number: 350
trajectory number: 360
trajectory number: 370
trajectory number: 380
trajectory number: 390
trajectory number: 400
trajectory number: 410
trajectory number: 420
trajectory number: 430

In [0]:
class state_predictor(nn.Module):
  def __init__(self,input_size, hidden_state_size,output_size):
    super(state_predictor, self).__init__()
    
    self.lstm=nn.LSTM(input_size,hidden_state_size)
    self.linear=nn.Linear(hidden_state_size, output_size)

  def forward(self,x,h):
    # h: hidden_state, c=output
    # x= x.view(batch_size,timesteps,embed_size)
    #print(h[0].shape)
    lstm_out,(h,c)=self.lstm(x,h)
    #print(out.size())
    #(batch_size*timesteps, hidden_size)
    #out.size(0):batch_size; out.size(1):timesteps, out.size(2): hidden_size
    fc_input=lstm_out.reshape(lstm_out.size(0)*lstm_out.size(1),lstm_out.size(2)) # flattens the matrix
    out = self.linear(fc_input)
    out = out.reshape(lstm_out.size(0), lstm_out.size(1), out.size(1))
    return out, (h,c)

In [0]:
num_layers = 1 
num_directions = 1

input_dim = 2 + n_actions
timesteps = 5
batch_size = 10
hidden_dim = 1024
output_dim = 2
learning_rate = 0.002
model = state_predictor(input_dim,hidden_dim, output_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.MSELoss()


In [48]:
x_train, y_train = full_env_train
print(x_train.dtype)
for it in range(20):
    for i in range(0,len(x_train), timesteps*batch_size):
        if len(x_train) > i + (timesteps*batch_size):
            x = x_train[i:i+(timesteps*batch_size),:]
            y = y_train[i:i+(timesteps*batch_size),:]
    
            inputs = torch.tensor(x)
            inputs = inputs.view(timesteps,batch_size,-1)
    
            targets = torch.tensor(y)
            targets = targets.view(timesteps,batch_size,-1)
    
            states = (torch.zeros(num_directions*num_layers,batch_size,hidden_dim),
                    torch.zeros(num_directions*num_layers,batch_size,hidden_dim))

            outputs,_ = model(inputs, states)
            #print('outputs:',outputs.size())
            #print('targets:',targets.size())
            loss = loss_fn(outputs, targets)

            model.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(),0.1)
            optimizer.step()
        
    print("Epoch [{}], Loss: {:.4f}".format(it, loss.item()))

float32
Epoch [0], Loss: 1.2235
Epoch [1], Loss: 0.9159
Epoch [2], Loss: 2.0151
Epoch [3], Loss: 1.1083
Epoch [4], Loss: 0.8947
Epoch [5], Loss: 0.7962
Epoch [6], Loss: 0.6352
Epoch [7], Loss: 0.8056
Epoch [8], Loss: 0.6347
Epoch [9], Loss: 0.7791
Epoch [10], Loss: 0.4550
Epoch [11], Loss: 0.5723
Epoch [12], Loss: 0.3145
Epoch [13], Loss: 0.4113
Epoch [14], Loss: 0.3419
Epoch [15], Loss: 0.7783
Epoch [16], Loss: 0.6551
Epoch [17], Loss: 0.3729
Epoch [18], Loss: 0.3188
Epoch [19], Loss: 0.4349


In [52]:
batch_size = 3
test_inputs = torch.tensor(x_train[0:timesteps*batch_size,:])
test_inputs = test_inputs.view(timesteps,batch_size,-1)
print(test_inputs.size())

targets = torch.tensor(y_train[0:timesteps*batch_size,:])
targets = targets.view(timesteps,batch_size,-1)
print(targets.size())
    
new_states = (torch.zeros(num_layers*num_directions,batch_size,hidden_dim),
          torch.zeros(num_directions*num_layers,batch_size,hidden_dim))

outputs,_ = model(test_inputs, new_states)
print('outputs:',outputs)
print('targets:',targets)

torch.Size([5, 3, 6])
torch.Size([5, 3, 2])
outputs: tensor([[[ 3.3684e+00,  1.2121e-02],
         [ 3.3787e+00,  9.9735e-04],
         [-2.2580e+00,  4.9191e-02]],

        [[-2.3351e+00,  1.1404e-01],
         [-2.3247e+00,  1.0857e-01],
         [ 3.3961e+00,  1.0153e-01]],

        [[ 3.5075e+00,  2.2878e-02],
         [ 3.4864e+00,  2.0179e-02],
         [-2.2165e+00,  7.6310e-02]],

        [[-2.3254e+00,  1.0348e-01],
         [ 3.3554e+00, -1.4667e-02],
         [ 3.4862e+00,  8.1130e-02]],

        [[ 3.4652e+00,  1.0515e-02],
         [ 3.4121e+00, -1.9552e-02],
         [-2.3378e+00, -4.0889e-02]]], grad_fn=<ViewBackward>)
targets: tensor([[[ 3.,  0.],
         [ 3.,  0.],
         [-2.,  0.]],

        [[-2.,  0.],
         [-3.,  0.],
         [ 3.,  0.]],

        [[ 3.,  0.],
         [ 3.,  0.],
         [-3.,  0.]],

        [[-3.,  0.],
         [ 3.,  0.],
         [ 3.,  0.]],

        [[ 3.,  0.],
         [ 3.,  0.],
         [-2.,  0.]]])


# Hyperparameters

In [0]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")#,monitor_gym=True)
ENV_NAME = 'MsPacman-ramDeterministic-v4' #'MsPacmanNoFrameskip-v4'  #'MsPacmanDeterministic-v4' # 'MsPacmanNoFrameskip-v4'
 
# Main loop hyperp
env_model_pretrain = False
pretrained_env_model = '/content/gdrive/My Drive/MsPacman-data/Env model/Env_model_21_4_2020-8_37.pt'#'/content/gdrive/My Drive/MsPacman-data/Env model/Env_model_17_4_2020-11_0.pt' #'/content/Env_model_17_4_2020-10_32.pt' #'/content/Epoch_no_43_Env_model_13_4_2020-8_22_checkpoint.pt'
ENV_LEARNING_RATE = 1e-4

AGGR_ITER = 10
STEPS_PER_AGGR = 1000

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 3000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Supervised Model Hyperp
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  10000

# Controller Hyperp
HORIZON_LENGTH = 1
NUM_ACTIONS_SEQUENCES = 1

load_data = True
if load_data:
    loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj_allowed_move_prev_action_fix.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-28_4_2020-18_0.npy' #'/content/gdrive/My Drive/MsPacman-data/only_agent_position_3000_traj_rand+allowed_moves.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-21_4_2020-7_47.npy'#'/content/gdrive/My Drive/MsPacman-data/Random trajectories/No_wall_movements/rand_traj_3000-_all_lives_16_4_2020-14_33.npy'#'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-16_4_2020-14_33.npy'#

collect_data = False

observation_channels = 1
action_dim = 1
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)

possible_positions = np.load('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0

if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)

cuda


In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR

config.train_model_iter = TRAIN_ITER_MODEL
config.env_lr = ENV_LEARNING_RATE
config.env_model_pretrain = env_model_pretrain
config.pretrained_env =  pretrained_env_model

config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.random_seed = random_seed

if load_data:
    config.loaded_trajectories = loaded_trajectories

# Main - Model training

Data collection and processing

In [0]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,n_actions,possible_positions)
if load_data:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
    filename = '/content/gdrive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
    np.save(filename,rand_dataset)

#rand_dataset = np.array([x for x in rand_dataset if not same(x)])
#rand_dataset = rand_dataset[0:100,:]
empty_dataset = []
env_train_data, env_val_data = data_collector.collate_data(rand_dataset,empty_dataset)

norm_env_train_data, norm_env_val_data = data_collector.setup_dataset(env_train_data, env_val_data)

number random examples: 893074 len(D_train_rand) 714460 len(D_valid_rand) 178614
